In [1]:
import os
import tensorflow as tf

## Create the model

In [12]:
# input is an image in BGR format of size 300x300x3 of type uint8
IMG_SHAPE = (300, 300, 3)
img_tensor = tf.keras.layers.Input(shape=IMG_SHAPE, name = 'bgr_uint8_image', dtype=tf.uint8)
x = tf.keras.backend.reverse(img_tensor, -1) # BGR to RGB
x = tf.keras.backend.cast(x, dtype=tf.float32) # uint8 to float32
x = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./127.5, offset=-1, trainable=False)(x)
x = tf.keras.layers.experimental.preprocessing.Resizing(160,160, trainable=False)(x)

# Create the base model from the pre-trained model MobileNet V2
model = tf.keras.applications.MobileNetV2(input_tensor=x,
                                               input_shape=(160,160,3),
                                               include_top=False,
                                               weights='imagenet')

# add the prediction layer at the top of the model
out_relu = model.output
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(out_relu)
prediction_layer = tf.keras.layers.Dense(1, name="prediction")(global_average_layer)
model = tf.keras.Model(inputs=img_tensor, outputs=prediction_layer, name="my_mobilenetv2")

In [13]:
model.summary()

Model: "my_mobilenetv2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bgr_uint8_image (InputLayer)    [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_ReverseV2_4 (Tensor [(None, 300, 300, 3) 0           bgr_uint8_image[0][0]            
__________________________________________________________________________________________________
tf_op_layer_Cast_4 (TensorFlowO [(None, 300, 300, 3) 0           tf_op_layer_ReverseV2_4[0][0]    
__________________________________________________________________________________________________
rescaling_4 (Rescaling)         (None, 300, 300, 3)  0           tf_op_layer_Cast_4[0][0]         
_____________________________________________________________________________________

## Save the model

In [ ]:
os.makedirs('saved_model', exist_ok=True)
model.save('saved_model/mobilenetv2')